<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/6_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 全 VARCHAR 的潛在問題

### 1. 資料型別不明確 → 易出錯

| 問題類型   | 範例                                 |
| ------ | ---------------------------------- |
| 錯誤排序   | `'10' < '2'` → 文字排序與數字排序不同         |
| 無法數值運算 | `SUM('123')` 不會報錯，但會很危險（隱式轉型）或直接失敗 |
| 濫用欄位   | 本來是日期，但有人塞「unknown」進去也不會錯，造成資料污染   |


👉 建議：明確類型應使用正確型別（如 INTEGER, DATE, TIMESTAMP）

### 2. 索引效率較差

- 對 VARCHAR（尤其長欄位）建立 index，效率會低於固定長度欄位。

- 比對成本高、排序效率差，甚至佔更多 index page 空間。

👉 建議：關鍵查詢欄位（如 ID、狀態碼）使用 CHAR(n) 或 INTEGER 更快

In [ ]:
CREATE TABLE COMMENT (
    ISSUE_NUMBER     VARCHAR(32)      NOT NULL,      # -- ID，保留 VARCHAR
    REVIEW_MONTH     CHAR(6)          NOT NULL,    # -- 格式建議 YYYYMM，故用 CHAR(6)
    CHART_NAME       VARCHAR(128)     NOT NULL,     # -- 圖表名稱，可考慮縮短上限
    PRODGRP_ID       VARCHAR(32)      NOT NULL,     # -- 產品群組 ID，保留 VARCHAR
    CATEGORY         CHAR(3)          NOT NULL,     # -- 若是代碼（如 HBT、SF），用固定長度
    FAB              CHAR(4)          NOT NULL,     # -- 廠別名稱（如 F12），用 CHAR
    PARAM_ID         VARCHAR(32)      NOT NULL,     # -- 參數名稱
    CHART_GROUP      VARCHAR(64),                   # -- 分群 ID，自由長度
    BASE_ON          CHAR(3)          NOT NULL,     # -- 判定依據，如 AVG、MAX...
    STAT             CHAR(3)          NOT NULL,     # -- 統計方法（如 MS、SD）
    CAUSE_CATEGORY   VARCHAR(64),                   # -- 原因分類
    TID_COMMENT      VARCHAR(2048),                          # -- 可大量文字，建議改 CLOB
    TID_ACTION       VARCHAR(128),                  # -- 動作說明
    REFER_FILE_1     VARCHAR(128),                  # -- 檔案名稱，128 通常夠用
    REFER_FILE_2     VARCHAR(128),
    REFER_FILE_3     VARCHAR(128),
    TID_USER         VARCHAR(32),                   # -- 提交者
    TID_UPDATE_TIME  TIMESTAMP   NULL,      # -- 修改時間 → 用時間型別
    QCE_COMMENT      VARCHAR(2048),                          # -- 同上
    QCE_ACTION       VARCHAR(2048),                          # -- 同上
    QCE_USER         VARCHAR(32),                   # -- 品管確認者
    QCE_UPDATE_TIME  TIMESTAMP NULL,      # -- 確認時間
    CREATE_TIME      TIMESTAMP       NOT NULL,      # -- 建立時間
    TRACKING         VARCHAR(512),                  # -- 補充追蹤說明
    HISTORY_FLAG     CHAR(1)         NOT NULL,      # -- 0 或 1，布林旗標
    CONSTRAINT pk_id PRIMARY KEY (
        ISSUE_NUMBER, REVIEW_MONTH, CHART_NAME,
        BASE_ON, STAT, TID_UPDATE_TIME, QCE_UPDATE_TIME
    )
);

## PRIMARY KEY (ISSUE_NUMBER, TID_UPDATE_TIME, QCE_UPDATE_TIME)

## 一次只會更新 一個 TID/QCE _ TIME 可以 TIMESTAMP ???

### Q : 一次只會更新 一個 TID/QCE _ TIME 可以使用 TIMESTAMP ???

- 只更新其中一個時間欄位，另一個欄位會保持原樣（不變）

- insert : 可以自己填入像 '2025-06-01 10:00:00' 這樣的字串，只要它格式符合 IBM Db2 TIMESTAMP 的標準格式，Db2 會自動幫你轉成 TIMESTAMP 型態存入。
(只要UPDATE語句沒有針對某欄位更新，該欄位就不會改變。)